In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.8"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.8

In [2]:
using System;
using Azure.AI.OpenAI;

In [3]:
var AZURE_OPENAI_ENDPOINT = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
var AZURE_OPENAI_KEY = Environment.GetEnvironmentVariable("AZURE_OPENAI_KEY");
var AZURE_OPENAI_DEPLOYMENTID = Environment.GetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENTGPT35");

In [4]:
var endpoint = new Uri(AZURE_OPENAI_ENDPOINT);
var credentials = new Azure.AzureKeyCredential(AZURE_OPENAI_KEY);
var openAIClient = new OpenAIClient(endpoint, credentials);

In [5]:
var systemPrompt = "You are a virtual agent that helps users write SQL queries using table schemas provided by the user.";

var userPrompt = 
    """
    ### Microsoft SQL tables, with their properties:
    #
    # Employee(id, name, department_id)
    # Department(id, name, address)
    # Salary_Payments(id, employee_id, amount, date)
    #
    ### A query to list the names of the departments which employed more than 10 employees in the last 3 months
    """;

In [6]:
var completionOptions = new ChatCompletionsOptions
{
    MaxTokens=150,
    Temperature=0f,
    FrequencyPenalty=0.0f,
    PresencePenalty=0.0f,
    NucleusSamplingFactor=1, // Top P
    StopSequences={"#",";"}    
};

In [7]:
completionOptions.Messages.Add(new ChatMessage(ChatRole.System, systemPrompt));

In [8]:
completionOptions.Messages.Add(new ChatMessage(ChatRole.User, userPrompt));

In [9]:
ChatCompletions response = await openAIClient.GetChatCompletionsAsync(AZURE_OPENAI_DEPLOYMENTID,completionOptions);

In [10]:
response.Choices.First().Message.Content

Here's the SQL query to list the names of the departments which employed more than 10 employees in the last 3 months:

```
SELECT d.name
FROM Department d
INNER JOIN Employee e ON d.id = e.department_id
INNER JOIN Salary_Payments sp ON e.id = sp.employee_id
WHERE sp.date >= DATEADD(month, -3, GETDATE())
GROUP BY d.name
HAVING COUNT(DISTINCT e.id) > 10